In [57]:
import pandas as pd
import itertools

In [2]:
data = pd.read_csv("COVID19MEXICO.csv")
data

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,RESULTADO_PCR,RESULTADO_PCR_COINFECCION,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL_COVID,CLASIFICACION_FINAL_FLU,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2025-07-29,167f1a,1,12,1,2,1,1,3,1,...,997,997,2,97,6,6,99,México,97,97
1,2025-07-29,ga8a474,1,4,20,2,20,20,413,1,...,997,997,2,97,6,6,99,México,97,97
2,2025-07-29,gb733da,1,6,8,2,8,8,37,1,...,997,997,2,97,6,6,99,México,97,97
3,2025-07-29,g9ff7b3,1,4,32,1,32,32,17,2,...,5,5,2,97,7,7,99,México,97,2
4,2025-07-29,g90b5c8,1,6,10,1,10,10,5,2,...,5,5,2,97,7,7,99,México,97,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99521,2025-07-29,g897fca,1,3,21,1,21,21,53,2,...,999,999,2,97,6,6,99,México,97,2
99522,2025-07-29,ge40110,1,15,16,2,15,16,102,2,...,999,999,2,97,6,6,99,México,97,2
99523,2025-07-29,g528417,1,15,2,1,2,2,1,2,...,999,999,2,97,6,6,99,México,97,2
99524,2025-07-29,gab8b98,1,12,19,2,19,19,44,1,...,999,999,2,97,6,6,99,México,97,97


# Variables que voy a tomar y que significa:
En el contexto de la base de datos de la Secretaría de Salud de México, los códigos de valor son:


1: Sí


2: No


97, 98, 99: Desconocido o no aplica


Edad:    Numero de la edad del paciente


NEUMONIA:	Paciente que presentó o no diagnóstico de neumonía.


DIABETES:	Paciente con un diagnóstico previo de diabetes.


EPOC:	Paciente con un diagnóstico previo de enfermedad pulmonar obstructiva crónica.


ASMA:	Paciente con un diagnóstico previo de asma.


INMUSUPR:	Paciente con un diagnóstico previo de inmunosupresión.


HIPERTENSION:	Paciente con un diagnóstico previo de hipertensión.


CARDIOVASCULAR:	Paciente con un diagnóstico previo de enfermedad cardiovascular.


OBESIDAD:	Paciente con un diagnóstico previo de obesidad.


RENAL_CRONICA:	Paciente con un diagnóstico previo de enfermedad renal crónica.


TABAQUISMO:	Paciente con un diagnóstico previo de tabaquismo.


OTRO_CASO:	Indica si el paciente tuvo contacto con otro caso confirmado de COVID-19.


FECHA_DEF: Contiene la fecha del fallecimiento del paciente en caso de tener 9999-99-99 o algo parecido no ha muerto



Y la más Importante


### CLASIFICACION_FINAL_COVID


3: Negativo a SARS-CoV-2: El caso fue descartado tras obtener un resultado de laboratorio negativo.


4: Caso Descartado por Dictaminación: El caso fue descartado por un comité de expertos sin una prueba de laboratorio concluyente.


5: Sospechoso: Es el estado inicial y transitorio de un caso, cuya clasificación final está pendiente de confirmación o descarte.


6: Confirmado por Laboratorio: El caso fue confirmado mediante una prueba de laboratorio, típicamente RT-PCR, el método de mayor fiabilidad.


7: Positivo por Dictaminación o Asociación: El caso fue catalogado como positivo por un comité de expertos basándose en un nexo epidemiológico con un caso confirmado o en criterios clínicos, sin una prueba de laboratorio concluyente.

### CLASIFICACION_FINAL_FLU

(3): Caso confirmado


(7): Caso negativo


(4, 5 ó 6): Sin información (sin muestra o muestra inválida)

# Selección de variables y limpieza

In [3]:
# Seleccionando las columnas de interes
data = data[[
    "EDAD",
    "NEUMONIA",
    "DIABETES",
    "EPOC",
    "ASMA",
    "INMUSUPR",
    "HIPERTENSION",
    "CARDIOVASCULAR",
    "OBESIDAD",
    "RENAL_CRONICA",
    "TABAQUISMO",
    "OTRO_CASO",
    "FECHA_DEF",
    "CLASIFICACION_FINAL_COVID",
    "CLASIFICACION_FINAL_FLU"
]].copy()
# Lista de valores que representan datos faltantes
missing_codes = [97, 99, 997, 999, 9999, '97', '99', '997', '999', '9999', '9999-99-99']

# Reemplazar por 0 en todo el DataFrame
data = data.replace(missing_codes, 0)

# Crear una columana adulto mayor para centrarse solo en esa la probabilidad 
adulto_mayor = data["EDAD"].apply(
    lambda x: 1 if pd.notna(x) and x >= 60 else 2) #se toma 60 que es la edad para ser adulto mayor en México
data.insert(1, 'AM', adulto_mayor)

# Esta funcion es para en lugar de tener fecha de muerte, tener si se murio o no
def convertir_no_cero_a_uno(valor):
    """Convierte valores no cero a 1, mantiene ceros como 0."""
    return 1 if valor != 0 else 2

muertos = data["FECHA_DEF"]
data["FECHA_DEF"] = muertos.apply(convertir_no_cero_a_uno)

def crear_columna_enfermo(df):
    df = df.copy()
    
    df['ENFERMO'] = (
        (df['CLASIFICACION_FINAL_COVID'].isin([6, 7])) |  # COVID positivo
        (df['CLASIFICACION_FINAL_FLU'] == 3)              # Influenza positivo
    )
    # Igual usamos 1 para casos positivos y 2 para negativos
    df['ENFERMO'] = df['ENFERMO'].map({True: 1, False: 2})
    
    return df
data = crear_columna_enfermo(data)

data

,EDAD,AM,NEUMONIA,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,FECHA_DEF,CLASIFICACION_FINAL_COVID,CLASIFICACION_FINAL_FLU,ENFERMO
0,8,2,2,2,2,2,2,2,2,2,2,2,2,2,6,6,1
1,23,2,2,2,2,2,2,2,2,2,2,2,2,2,6,6,1
2,18,2,2,2,2,1,2,2,2,2,2,2,2,2,6,6,1
3,24,2,2,2,2,1,2,2,2,2,2,2,2,2,7,7,1
4,47,2,2,2,2,2,2,2,2,2,2,2,2,2,7,7,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99521,83,1,2,2,2,2,2,1,2,2,2,2,2,2,6,6,1
99522,50,2,1,1,2,2,2,1,2,2,1,2,1,2,6,6,1
99523,13,2,2,2,2,2,2,2,2,2,2,2,2,2,6,6,1
99524,37,2,2,2,2,2,2,2,2,2,2,2,1,2,6,6,1


# Probabilidades de estar enfermo dado el sintoma

In [4]:
s = data["ENFERMO"].value_counts()
s

ENFERMO
1    92054
2     7472
Name: count, dtype: int64

Ver cuantos muertos hay y que % representan en el dataset

Ver todos los muertos que no tienen sintomas o no sabemos si los tienen

In [14]:
sintomas = ["AM", "NEUMONIA", "DIABETES", "EPOC", "ASMA", 
            "INMUSUPR", "HIPERTENSION", "CARDIOVASCULAR", 
            "OBESIDAD", "RENAL_CRONICA", "TABAQUISMO", "OTRO_CASO"]

# Filtrar pacientes muertos
muertos = data[data['FECHA_DEF'] == 1]

# Contar cuántos tienen todos los síntomas = 0
todos_0 = muertos[(muertos[sintomas] == 0).all(axis=1)]
num_todos_0 = len(todos_0)

# Contar cuántos tienen todos los síntomas = 2
todos_2 = muertos[(muertos[sintomas] == 2).all(axis=1)]
num_todos_2 = len(todos_2)

print(f"Muertos con todos los síntomas = 0: {num_todos_0}")
print(f"Muertos con todos los síntomas = 2: {num_todos_2}")

Muertos con todos los síntomas = 0: 0
Muertos con todos los síntomas = 2: 279


In [18]:
muertos = 0
for i in range(len(data["FECHA_DEF"])):
    if data["FECHA_DEF"][i] == 1:
        muertos +=1
print(muertos)
print(f"{muertos*100/99526}%")

3710
3.7276691517794345%


Aca estoy viendo sin calcular bayes como estan mas o menos las relaciones entre sintomas y el estar enfermo

In [6]:
columnas = ["AM", "NEUMONIA", "DIABETES", "EPOC", "ASMA", "INMUSUPR", "HIPERTENSION", "CARDIOVASCULAR", "OBESIDAD", "RENAL_CRONICA", "TABAQUISMO", "OTRO_CASO"]
for sintoma in columnas:
    # Contar cuantas veces coincide el sintoma con que este enfermo el paciente
    sintoma_y_enfermo = len(data[(data[sintoma] == 1) & (data['ENFERMO'] == 1)])
    total_con_sintoma = len(data[data[sintoma] == 1])
    
    #Calcular cuantas personas enfermas tienen el sintoma
    pct_enfermos_con_sintoma = (sintoma_y_enfermo / 92054) * 100
    # Calcular de las personas con el sintoma cuantos estan enfermos
    pct_sintoma_que_son_enfermos = (sintoma_y_enfermo / total_con_sintoma) * 100
        
    print(f"{sintoma:15} {pct_enfermos_con_sintoma:.2f}% {pct_sintoma_que_son_enfermos:10.2f}%")

AM              21.29%      91.15%
NEUMONIA        23.73%      92.90%
DIABETES        13.10%      91.69%
EPOC            3.36%      92.46%
ASMA            3.97%      93.23%
INMUSUPR        3.07%      89.64%
HIPERTENSION    15.80%      91.34%
CARDIOVASCULAR  3.36%      92.25%
OBESIDAD        6.74%      92.09%
RENAL_CRONICA   3.43%      91.24%
TABAQUISMO      4.49%      92.09%
OTRO_CASO       18.23%      90.45%


Aca estoy viendo la relacion entre los sintomas y la muerte para quedarme con el top 5 sintomas más mortiferos

In [7]:
for sintoma in columnas:
    sintoma_y_muerto = len(data[(data[sintoma] == 1) & (data['FECHA_DEF'] == 1)])
    total_con_sintoma = len(data[data[sintoma] == 1])
    
    #Calcular cuantas personas muertas tenian el sintoma
    pct_muertos_con_sintoma = (sintoma_y_muerto / 92054) * 100
    # Calcular de las personas con el sintoma cuantos estan muertos
    pct_sintoma_que_estanmuertos = (sintoma_y_muerto / total_con_sintoma) * 100
        
    print(f"{sintoma:15} {pct_muertos_con_sintoma:.2f}% {pct_sintoma_que_estanmuertos:10.2f}%")

AM              2.33%      10.00%
NEUMONIA        2.64%      10.32%
DIABETES        1.36%       9.49%
EPOC            0.44%      12.20%
ASMA            0.10%       2.25%
INMUSUPR        0.28%       8.08%
HIPERTENSION    1.54%       8.93%
CARDIOVASCULAR  0.46%      12.49%
OBESIDAD        0.46%       6.33%
RENAL_CRONICA   0.46%      12.35%
TABAQUISMO      0.43%       8.80%
OTRO_CASO       0.39%       1.94%


Aca dejo de donde se sacaron las 5 probabilidades y como se obtienen


AM = https://www.gob.mx/inapam/articulos/proyecciones-demograficas-de-un-mexico-que-envejece


Neumonia = No se encontraron datos exactos, se va a tener que calcular en base a los datos obtenidos


EPOC = https://www.gob.mx/salud/articulos/sabias-que-10-de-la-poblacion-mexicana-padece-epoc


CARDIOVASCULAR =  No se encontraron datos exactos, se va a tener que calcular en base a los datos obtenidos


RENAL CRONICA = https://www.gaceta.unam.mx/casi-12-de-la-poblacion-sufre-enfermedad-renal-cronica-en-mexico/



In [8]:
def calculo_sintoma(df,sintoma):
    total_con_sintoma = len(df[df[sintoma] == 1])
    Pct_Sintoma = (total_con_sintoma/99526)*100
    return Pct_Sintoma

In [9]:
mortiferos = ["AM", "NEUMONIA", "EPOC", "CARDIOVASCULAR", "RENAL_CRONICA"]
Probabilidad_AM = 12.8
Probabilidad_Neumonia = round(calculo_sintoma(data,"NEUMONIA"),2) #23.63
Probabilidad_EPOC = 10
Probabilidad_Cardiovascular = round(calculo_sintoma(data,"CARDIOVASCULAR"),2) #3.37
Probabilidad_Renal_Cronica = 12

Ya tenemos P(Si) = Probabilidad de tener un sintoma. Abajo coloco P(D|Si) = Probabilidad de tener el sintoma dado que el diagnostico es positivo para cada sintoma. P(Si|D) = Probabilidad de estar enfermo dado que tienes un sintoma, hay que calculuar P(D)

In [10]:
Pct_de_enfermos_con_AM = 21.29 
Pct_degente_con_Am_Enfermos = 91.15
Pct_de_enfermos_con_Neumonia = 23.73
Pct_degente_con_Neumonia_Enfermos = 92.90
Pct_de_enfermos_con_EPOC = 3.36
Pct_degente_con_EPOC_Enfermos = 92.46
Pct_de_enfermos_con_Cardiovascular = 3.36
Pct_degente_con_Cardiovascular_Enfermos = 92.25
Pct_de_enfermos_con_Renal_Cronica = 3.43
Pct_degente_con_Renal_Cronica_Enfermos = 91.24

Aca voy a unir todo a una df para manipularlo más facilmente

In [11]:
Ndata = {
    'Sintoma': ["AM", "NEUMONIA", "EPOC", "CARDIOVASCULAR", "RENAL_CRONICA"],
    'Probabilidad_Enfermedad': [12.8, 23.63, 10.0, 3.37, 12.0],
    'Pct_de_enfermos_con_sintoma': [21.29, 23.73, 3.36, 3.36, 3.43],
    'Pct_de_gente_con_sintoma_Enfermos': [91.15, 92.90, 92.46, 92.25, 91.24]
}

dfNdata = pd.DataFrame(Ndata)
dfNdata

,Sintoma,Probabilidad_Enfermedad,Pct_de_enfermos_con_sintoma,Pct_de_gente_con_sintoma_Enfermos
0,AM,12.80,21.29,91.15
1,NEUMONIA,23.63,23.73,92.90
2,EPOC,10.00,3.36,92.46
3,CARDIOVASCULAR,3.37,3.36,92.25
4,RENAL_CRONICA,12.00,3.43,91.24


## Vamos a hallar P(D)

Que es la probabilidad de diagnostico

In [12]:
# Condición booleana: al menos un síntoma mortífero
filtro_mortiferos = data[mortiferos].any(axis=1)

# Número de pacientes con al menos un síntoma mortífero
num_pacientes = filtro_mortiferos.sum()
print(f"Número de pacientes con al menos un síntoma: {num_pacientes}")

# Condición booleana: pacientes enfermos
filtro_enfermos = data['ENFERMO'] == 1

# Combinar ambas condiciones
pacientes_mortiferos_enfermos = data[filtro_mortiferos & filtro_enfermos]

# Contar cuántos cumplen ambas condiciones
num_pacientes_enfermos = len(pacientes_mortiferos_enfermos)
print(f"Número de pacientes con al menos un sintoma y que están enfermos: {num_pacientes_enfermos}")
p_D = round((num_pacientes_enfermos/num_pacientes)*100,2)
print(f"P(D) = {p_D}%")


Número de pacientes con al menos un síntoma: 99526
Número de pacientes con al menos un sintoma y que están enfermos: 92054
P(D) = 92.49%


Ahora hay que calcular $P(D \mid \text{AM, NEUMONIA, EPOC, CARDIOVASCULAR, RENALCRONICA})$ : 

Al tener 5 sintomas vamos a tener 2^5 combinaciones de diagnostico


### Calcular P(D | combinación de síntomas)

Asumiendo **independencia de los síntomas (Naive Bayes)**:

$$
P(D \mid S_1,...,S_5) = \frac{P(D) \prod_i P(S_i|D)^{S_i} (1-P(S_i|D))^{1-S_i}}{P(D) \prod_i P(S_i|D)^{S_i} (1-P(S_i|D))^{1-S_i} + (1-P(D)) \prod_i P(S_i|\neg D)^{S_i} (1-P(S_i|\neg D))^{1-S_i}}
$$

Donde:

$$
P(S_i|\neg D) = \frac{P(S_i) - P(S_i|D)\cdot P(D)}{1-P(D)}
$$

* $S_i = 1$ si el síntoma está presente, 0 si no.
* La fórmula normaliza la probabilidad entre **D** y **¬D**.



In [ ]:
no_sintomas_y_muerto = len(data[
    ((data['AM'] == 0) | (data['AM'] == 2)) &
    ((data['NEUMONIA'] == 0) | (data['NEUMONIA'] == 2)) &
    ((data['EPOC'] == 0) | (data['EPOC'] == 2)) &
    ((data['CARDIOVASCULAR'] == 0) | (data['CARDIOVASCULAR'] == 2)) &
    ((data['RENAL_CRONICA'] == 0) | (data['RENAL_CRONICA'] == 2)) &
    (data['FECHA_DEF'] == 1)
]) -  279 # se quitan 279 que son los muertos asintomaticos calculado anteriormente

print(f"Número de pacientes que no tenían ningun sintoma de los seleccionados y murieron: {no_sintomas_y_muerto}")
pct_muertos_faltantes = round((((no_sintomas_y_muerto)/muertos)*100) ,2)
print(f"Muertos que no cubren los sintomas: {pct_muertos_faltantes}%")

Número de pacientes que no tenían ningun sintoma de los seleccionados y murieron: 498
Muertos que no cubren los sintomas: 13.42%


In [54]:
sintomas_y_muerto = len(data[
    (data['AM'] != 1) &
    (data['NEUMONIA'] != 1) &
    (data['EPOC'] != 1)  &
    (data['CARDIOVASCULAR'] != 1) &
    (data['RENAL_CRONICA'] != 1) &
    (data['FECHA_DEF'] == 1)
])
print(f"Número de pacientes que tenían sintoma(s) de los seleccionados y murieron: {sintomas_y_muerto}")

Número de pacientes que tenían sintoma(s) de los seleccionados y murieron: 499


In [ ]:
resultados = []

# Primero calculamos el total de muertes para los porcentajes
total_muertes = len(data[data['FECHA_DEF'] == 1])

for comb in itertools.product([0, 1], repeat=len(mortiferos)):
    # Construir la query string
    condiciones_query = []
    for i, col in enumerate(mortiferos):
        if comb[i] == 1:
            condiciones_query.append(f"{col} == 1")
        else:
            condiciones_query.append(f"{col} != 1")
    
    query_str = " & ".join(condiciones_query) + " & FECHA_DEF == 1"
    
    # Calcular el conteo
    conteo = len(data.query(query_str))
    
    # Calcular porcentaje
    porcentaje = (conteo / total_muertes * 100) if total_muertes > 0 else 0
    
    # Crear diccionario con resultados
    resultado = {col: comb[i] for i, col in enumerate(columnas)}
    resultado['MUERTES'] = conteo
    resultado['PORCENTAJE'] = round(porcentaje, 2)
    resultados.append(resultado)

# Crear DataFrame final
tabla_resultados = pd.DataFrame(resultados)

print(tabla_resultados)
print(f"\nTotal de muertes: {total_muertes}")

    AM  NEUMONIA  EPOC  CARDIOVASCULAR  RENAL_CRONICA  MUERTES  PORCENTAJE
0    0         0     0               0              0      499          13
1    0         0     0               0              1       47           1
2    0         0     0               1              0       23           1
3    0         0     0               1              1        9           0
4    0         0     1               0              0       10           0
5    0         0     1               0              1        0           0
6    0         0     1               1              0        4           0
7    0         0     1               1              1        1           0
8    0         1     0               0              0      799          22
9    0         1     0               0              1       71           2
10   0         1     0               1              0       52           1
11   0         1     0               1              1        8           0
12   0         1     1   

In [62]:
resultados = []

# Primero calculamos el total de muertes para los porcentajes
total_enfermos = len(data[data['ENFERMO'] == 1])

for comb in itertools.product([0, 1], repeat=len(mortiferos)):
    # Construir la query string
    condiciones_query = []
    for i, col in enumerate(mortiferos):
        if comb[i] == 1:
            condiciones_query.append(f"{col} == 1")
        else:
            condiciones_query.append(f"{col} != 1")
    
    query_str = " & ".join(condiciones_query) + " & ENFERMO == 1"
    
    # Calcular el conteo
    conteo = len(data.query(query_str))
    
    # Calcular porcentaje
    porcentaje = (conteo / total_enfermos * 100) if total_enfermos > 0 else 0
    
    # Crear diccionario con resultados
    resultado = {col: comb[i] for i, col in enumerate(columnas)}
    resultado['ENFERMOS'] = conteo
    resultado['PORCENTAJE'] = round(porcentaje, 2)
    resultados.append(resultado)

# Crear DataFrame final
tabla_resultados = pd.DataFrame(resultados)

print(tabla_resultados)
print(f"\nTotal de enfermos: {total_enfermos}")

    AM  NEUMONIA  EPOC  CARDIOVASCULAR  RENAL_CRONICA  ENFERMOS  PORCENTAJE
0    0         0     0               0              0     56475          61
1    0         0     0               0              1       763           1
2    0         0     0               1              0       477           1
3    0         0     0               1              1        74           0
4    0         0     1               0              0       249           0
5    0         0     1               0              1        16           0
6    0         0     1               1              0        21           0
7    0         0     1               1              1        15           0
8    0         1     0               0              0     13107          14
9    0         1     0               0              1       508           1
10   0         1     0               1              0       414           0
11   0         1     0               1              1        69           0
12   0      

Las variables elejidas tienen el 39% de los enfermos pero el 87% de los muertos

Probabilidad de morir si estas por alguno(s) de los sintomas enfermo

In [65]:
Enfermos_de_sintomas_seleccionados = 92054 - 56475
pct_muertos_sintomas_seleccionados = 3710*0.87
pct_sintomas_seleccionados_que_estanmoridos = round((pct_muertos_sintomas_seleccionados/Enfermos_de_sintomas_seleccionados),2)
print(f"Las variables tienen {Enfermos_de_sintomas_seleccionados} peronas enfermas 39% del total, de los cuales {pct_muertos_sintomas_seleccionados} son los muertos siendo un 87% de todos los muertos. dejando la probabilidad de que si tienes alguno(s) de los 5 sintomas te mueras con un {pct_sintomas_seleccionados_que_estanmoridos*100}%")

Las variables tienen 35579 peronas enfermas 39% del total, de los cuales 3227.7 son los muertos siendo un 87% de todos los muertos. dejando la probabilidad de que si tienes alguno(s) de los 5 sintomas te mueras con un 9.0%
